In [ ]:
import numpy as np
import pandas as pd
!pip install geopandas
import geopandas as gpd
from matplotlib import pyplot as plt
from datetime import date
import os

# To avoid unnecessary warnings in the code
import warnings
warnings.filterwarnings('ignore')

     |████████████████████████████████| 931kB 2.6MB/s 
     |████████████████████████████████| 10.4MB 6.8MB/s 
     |████████████████████████████████| 14.7MB 334kB/s 


In [ ]:
try:
  from google.colab import drive
  IN_COLAB = True
except:
  IN_COLAB = False
  os.chdir("F:\My Drive\Shared\CS 498 Projects\Final Project\Data")
if IN_COLAB:
  drive.mount('/content/drive')
  os.chdir("/content/drive/My Drive/Shared/CS 498 Projects/Final Project/Data")
  os.listdir("/content/drive/My Drive/Shared/CS 498 Projects/Final Project/Data")
df_prep = pd.read_csv('Illinois/Cleaned_Weather/Daily_prep@IL_1950-2019_Apr-15_Oct-15.csv',
                      sep=',', index_col='Date',na_values=-99)
df_tmax = pd.read_csv('Illinois/Cleaned_Weather/Daily_tmax@IL_1950-2019_Apr-15_Oct-15.csv',
                      sep=',', index_col='Date',na_values=-99)
df_tmin = pd.read_csv('Illinois/Cleaned_Weather/Daily_tmin@IL_1950-2019_Apr-15_Oct-15.csv',
                      sep=',', index_col='Date',na_values=-99)
df_gdd = pd.read_csv('Illinois/Cleaned_Weather/Daily_gdd@IL_1950-2019_Apr-15_Oct-15.csv',
                      sep=',', index_col='Date',na_values=-99)

# Change the index 'Date' format to python datetime.date(yyyy,mm,dd) format
date_index = [date(int(d.split('/')[2]),
                   int(d.split('/')[1]),
                   int(d.split('/')[0])) 
              for d in df_prep.index]
df_prep.set_index(pd.Index(date_index),'Date', inplace=True)
df_tmax.set_index(pd.Index(date_index),'Date', inplace=True)
df_tmin.set_index(pd.Index(date_index),'Date', inplace=True)
df_gdd.set_index(pd.Index(date_index),'Date', inplace=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:

# Year  Counties SeasonalPrep    ....   ....
Y = [i for i in range(1950, 2020)]
Years = []
Counties = []
SeasonalPrep, SeasonalTmin, SeasonalTmax = [], [], []
SeasonalAvgTmin, SeasonalAvgTmax = [], []
WetDays, DaysA30, DaysB10, DaysB_2, GDD = [], [], [], [], []

for y in Y:
  Years.extend([y for i in range(len(list(df_prep.columns.values)))])
  Counties.extend(list(df_prep.columns.values))
  SeasonalPrep.extend(df_prep.loc[date(y,4,15):date(y,10,15),:].sum())
  SeasonalTmin.extend(df_tmin.loc[date(y,4,15):date(y,10,15),:].min())
  SeasonalTmax.extend(df_tmax.loc[date(y,4,15):date(y,10,15),:].max())
  SeasonalAvgTmin.extend(df_tmin.loc[date(y,4,15):date(y,10,15),:].mean())
  SeasonalAvgTmax.extend(df_tmax.loc[date(y,4,15):date(y,10,15),:].mean())
  WetDays.extend(list(df_prep.loc[date(y,4,15):date(y,10,15),:][df_prep.loc[date(y,4,15):date(y,10,15),:] > 1].count()))
  DaysA30.extend(list(df_tmax.loc[date(y,4,15):date(y,10,15),:][df_tmax.loc[date(y,4,15):date(y,10,15),:] > 30].count()))
  DaysB10.extend(list(df_tmin.loc[date(y,4,15):date(y,10,15),:][df_tmin.loc[date(y,4,15):date(y,10,15),:] < 10].count()))
  DaysB_2.extend(list(df_tmin.loc[date(y,4,15):date(y,10,15),:][df_tmin.loc[date(y,4,15):date(y,10,15),:] < -2].count()))
  GDD.extend(df_gdd.loc[date(y,4,15):date(y,10,15),:].sum())

df_Clim_Var = pd.DataFrame({'Year': Years,
                            'County': Counties,
                            'SeasonalPrep': SeasonalPrep,
                            'SeasonalTmax': SeasonalTmax,
                            'SeasonalTmin': SeasonalTmin,
                            'SeasonalAvgTmax': SeasonalAvgTmax,
                            'SeasonalAvgTmin': SeasonalAvgTmin,
                            'WetDays': WetDays,
                            'DaysA30': DaysA30,
                            'DaysB10': DaysB10,
                            'DaysB_2': DaysB_2,
                            'GDD': GDD})
# df_Clim_Var.to_csv('Illinois/Cleaned_Weather/Clim_Var@IL_1950-2019.csv')

In [ ]:
df_soybean = pd.read_csv('Illinois/Soybean_Yield_1925-2019.csv',
                      sep=',',na_values=-99)[['Year','County','Value']]
df_corn=pd.read_csv('Illinois/Corn_Yield_1925-2019.csv',
                      sep=',',na_values=-99)[['Year','County','Value']]
df_Clim_Var= pd.merge(df_Clim_Var, df_soybean, how='left', on=['Year', 'County'])
df_Clim_Var= pd.merge(df_Clim_Var, df_corn, how='left', on=['Year', 'County'],suffixes=['_soybean', '_corn'])
df_Clim_Var.to_csv('Illinois/Cleaned_Weather/all_data@IL_1950-2019.csv',index=False)

In [ ]:
df_Clim_Var.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7140 entries, 0 to 7139
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             7140 non-null   int64  
 1   County           7140 non-null   object 
 2   SeasonalPrep     7140 non-null   float64
 3   SeasonalTmax     7140 non-null   float64
 4   SeasonalTmin     7140 non-null   float64
 5   SeasonalAvgTmax  7140 non-null   float64
 6   SeasonalAvgTmin  7140 non-null   float64
 7   WetDays          7140 non-null   int64  
 8   DaysA30          7140 non-null   int64  
 9   DaysB10          7140 non-null   int64  
 10  DaysB_2          7140 non-null   int64  
 11  GDD              7140 non-null   float64
 12  Value_soybean    7037 non-null   float64
 13  Value_corn       7028 non-null   float64
dtypes: float64(8), int64(5), object(1)
memory usage: 836.7+ KB


In [ ]:
df_Clim_Var[df_Clim_Var["Value_soybean"].isnull()][["Year","County"]].values;

In [ ]:
df_Clim_Var[df_Clim_Var["Value_corn"].isnull()][["Year","County"]].values;

### Adding latitude and longitude of counties' centroid as columns

In [ ]:
gdf = gpd.read_file('Illinois/GIS Files/Illinois_Counties_GCS_WGS84.shp')
adjacent={}
Latitude=[]
Longitude=[]
for i in range(len(gdf.index)):
 Latitude.append(gdf.iloc[i].geometry.centroid.y)
 Longitude.append(gdf.iloc[i].geometry.centroid.x)
df_county_lat = pd.DataFrame({'Latitude': Latitude,'Longitude':Longitude},
                             index=gdf.iloc[:].COUNTY_NAM)

In [ ]:
df_county_lat['County'] = df_county_lat.index
df_Clim_Var=pd.merge(df_Clim_Var,df_county_lat)

df_Clim_Var.rename(columns = {'Value_soybean':'Soybean', 'Value_corn':'Corn'}, inplace=True)
df_Clim_Var.head()

,Year,County,SeasonalPrep,SeasonalTmax,SeasonalTmin,SeasonalAvgTmax,SeasonalAvgTmin,WetDays,DaysA30,DaysB10,DaysB_2,GDD,Soybean,Corn,Latitude,Longitude
0,1950,ADAMS,481.046667,35.023333,-0.991667,25.564627,13.324800,57,28,42,0,2708.500834,28.0,55.0,39.98785,-91.188437
1,1951,ADAMS,513.565000,36.751667,-2.735000,25.901047,14.245191,66,42,39,1,2792.585834,28.0,60.0,39.98785,-91.188437
2,1952,ADAMS,496.356667,38.963333,-3.333333,27.672204,14.533133,52,74,43,3,2917.287500,27.0,59.0,39.98785,-91.188437
3,1953,ADAMS,334.023333,38.981667,-3.941667,28.367377,14.761357,47,86,44,2,2947.965001,22.5,52.0,39.98785,-91.188437
4,1954,ADAMS,622.691667,43.956667,0.105000,28.192687,15.860492,67,82,35,0,3000.706667,25.5,46.0,39.98785,-91.188437


In [ ]:
col_name = df_Clim_Var.columns.to_list()
reorder_col_name = col_name[:2] + col_name[-2:] + col_name[-4:-2] + col_name[2:-4]
df_Clim_Var = df_Clim_Var[reorder_col_name]
df_Clim_Var.head()

,Year,County,Latitude,Longitude,Soybean,Corn,SeasonalPrep,SeasonalTmax,SeasonalTmin,SeasonalAvgTmax,SeasonalAvgTmin,WetDays,DaysA30,DaysB10,DaysB_2,GDD
0,1950,ADAMS,39.98785,-91.188437,28.0,55.0,481.046667,35.023333,-0.991667,25.564627,13.324800,57,28,42,0,2708.500834
1,1951,ADAMS,39.98785,-91.188437,28.0,60.0,513.565000,36.751667,-2.735000,25.901047,14.245191,66,42,39,1,2792.585834
2,1952,ADAMS,39.98785,-91.188437,27.0,59.0,496.356667,38.963333,-3.333333,27.672204,14.533133,52,74,43,3,2917.287500
3,1953,ADAMS,39.98785,-91.188437,22.5,52.0,334.023333,38.981667,-3.941667,28.367377,14.761357,47,86,44,2,2947.965001
4,1954,ADAMS,39.98785,-91.188437,25.5,46.0,622.691667,43.956667,0.105000,28.192687,15.860492,67,82,35,0,3000.706667


In [ ]:
df_Clim_Var.to_csv('Illinois/Cleaned_Weather/AnnualData_lat_lon@IL_1950-2019.csv',index=False)